In [30]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
import sys

sys.path.insert(0, '../NSCL-PyTorch-Release/')

In [32]:
import torch
from torch.utils.data import DataLoader

import json
from copy import deepcopy

from tqdm import tqdm_notebook as tqdm

from my_imp.models.nscl import NSCLModel
from my_imp.copied.scene_annotation import annotate_objects
from my_imp.data import MyDataset, gen_image_transform, gen_bbox_transform

In [33]:
img_size = 256

ans_dict_json = 'data/clevr/CLEVR_built_dictionaries.json'

image_root = 'data/clevr/train/images/'
vocab_json = 'data/clevr/train/vocab.json'
train_scenes_json = 'data/clevr/train/scenes.json'
train_questions_json = 'data/clevr/train/questions.json'

val_image_root = 'data/clevr/val/images/'
val_vocab_json = 'data/clevr/val/vocab.json'
val_scenes_json = 'data/clevr/val/scenes.json'
val_questions_json = 'data/clevr/val/questions.json'

dataset_args = {
    'train': {
        'scenes_json': train_scenes_json,
        'questions_json': train_questions_json,
        'image_root': image_root,
        'vocab_json': vocab_json,
    },
    'val': {
        'scenes_json': val_scenes_json,
        'questions_json': val_questions_json,
        'image_root': val_image_root,
        'vocab_json': val_vocab_json,
    },
}

image_transform = gen_image_transform(img_size)
bbox_transform = gen_bbox_transform(img_size)

In [29]:
split = 'train'

my_dataset = MyDataset(
    image_transform=image_transform,
    bbox_transform=bbox_transform,
    ans_dict_json=ans_dict_json,
    **dataset_args[split],
    # sample_size=100000,
    # seed=42,
)

TypeError: __init__() got an unexpected keyword argument 'sample_size'

In [28]:
my_dataset[0]

{'image': tensor([[[-0.2856, -0.3198, -0.3198,  ..., -0.3541, -0.3541, -0.3541],
          [-0.3198, -0.3027, -0.3027,  ..., -0.3712, -0.3541, -0.3541],
          [-0.3198, -0.3198, -0.3027,  ..., -0.3712, -0.3541, -0.3541],
          ...,
          [-0.1143, -0.1143, -0.0972,  ...,  0.6049,  0.6049,  0.6049],
          [-0.0972, -0.1143, -0.1143,  ...,  0.6221,  0.6049,  0.6221],
          [-0.1143, -0.0972, -0.1143,  ...,  0.6221,  0.6049,  0.6221]],
 
         [[-0.1625, -0.1975, -0.1975,  ..., -0.2500, -0.2500, -0.2325],
          [-0.1975, -0.1800, -0.1800,  ..., -0.2500, -0.2325, -0.2325],
          [-0.1975, -0.1975, -0.1800,  ..., -0.2500, -0.2325, -0.2325],
          ...,
          [-0.0049,  0.0126,  0.0126,  ...,  0.6954,  0.6954,  0.6954],
          [ 0.0126, -0.0049, -0.0049,  ...,  0.7129,  0.6954,  0.7129],
          [-0.0049,  0.0126,  0.0126,  ...,  0.7129,  0.6954,  0.7129]],
 
         [[ 0.0605,  0.0256,  0.0256,  ..., -0.0267, -0.0267, -0.0092],
          [ 0.0256,

In [24]:
len(my_dataset)

699989

In [19]:
from my_imp.data import collate_fn

loader = DataLoader(my_dataset, batch_size=64, num_workers=4, shuffle=True, collate_fn=collate_fn)

In [22]:
for _ in tqdm(loader):
    pass

KeyboardInterrupt: 

In [14]:
import torch
import torch.nn as nn

import numpy as np

In [65]:
class SNT(nn.Module):
    def __init__(self, N, dropout=0.5):
        super().__init__()
        
        self.visual_layers = None
        self.question_layers = None
        
        
    def forward(self, objects, questions):
        prev_objects = objects
        prev_questions = questions
        for v, q in zip(self.visual_layers, self.question_layers):
            objects = v(objects, prev_questions)
            questions = q(questions, objects)
            
            prev_objects = objects
            prev_questions = questions
      
    
class StandardBlock(nn.Module):
    def __init__(self,
                 dim=256,
                 dropout=0.5,
                 num_heads=8,
                ):
        super().__init__()
        
        self.intra_attention = nn.MultiheadAttention(
            embed_dim=dim,
            num_heads=num_heads,
            
        )
        self.inter_attention = None
        
        self._norm1 = nn.BatchNorm1d(dim)
        self._norm2 = nn.BatchNorm1d(dim)
        self._norm3 = nn.BatchNorm1d(dim)
            
    def forward(self, intra, inter):
        pass
            
class VisualBlock(nn.Module):
    def __init__(self, dim=256, dropout=0.5):
        super().__init__()
        
        self.mmha = nn.MultiheadAttention()
        
        self._norm1 = nn.BatchNorm1d(dim)
        self._norm2 = nn.BatchNorm1d(dim)
        self._norm3 = nn.BatchNorm1d(dim)
        
    def forward(self, objects, prev_question, mask=None):
        if mask is None:
            # TODO
            mask = None 
        
        
class QuestionBlock(nn.Module):
    def __init__(self, dim=256, dropout=0.5):
        super().__init__()
        
    def forward(self, question, prev_objects, mask=None):
        if mask is None:
            mask = None
        
       

In [67]:
batch = next(iter(loader))

In [72]:
nscl.resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [74]:
nscl = NSCLModel()
nscl(batch)

torch.Size([2, 98304])
Using /private/var/folders/3l/bh48rdbn141c_fvqn5vf6whh0000gn/T/torch_extensions as PyTorch extensions root...
Creating extension directory /private/var/folders/3l/bh48rdbn141c_fvqn5vf6whh0000gn/T/torch_extensions/_prroi_pooling...
Detected CUDA files, patching ldflags


OSError: CUDA_HOME environment variable is not set. Please set it to your CUDA install root.

In [39]:
from tqdm import tqdm_notebook as tqdm

In [ ]:
for batch in tqdm(loader):
    pass

In [32]:
print(batch[0][0].size())
print(batch[0][1].size())
print(batch[0][2].size())
print(batch[0][3].size())
print(batch[1].size())


torch.Size([2, 3, 256, 384])
torch.Size([2, 7, 4])
torch.Size([2])
torch.Size([2, 32])
torch.Size([2])


In [60]:
from nscl.datasets.datasets import NSCLDataset

In [78]:
NSCLDataset(val_scenes_json, val_questions_json, image_root=val_image_root, vocab_json=val_vocab_json, image_transform=img_transform)

10 23:18:09 Loading scenes from: "data/clevr/val/scenes.json".
10 23:18:11 Loading questions from: "data/clevr/val/questions.json".
Exception ignored in: <function _DataLoaderIter.__del__ at 0x126e03378>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 677, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 659, in _shutdown_workers
    w.join()
  File "/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 140, in join
    res = self._popen.wait(timeout)
  File "/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/popen_fork.py", line 48, in wait
    return self.poll(os.WNOHANG if timeout == 0.0 else 0)
  File "/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/popen_fork.py", lin

In [41]:
torch.cuda.is_available()

False